In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import copy
import time

# Global Variables

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 105.0.5195
Get LATEST chromedriver version for 105.0.5195 google-chrome
Driver [C:\Users\ADMIN\.wdm\drivers\chromedriver\win32\105.0.5195.52\chromedriver.exe] found in cache


In [ ]:
wait_time = 3

In [62]:
restaurants_df = pd.DataFrame(columns=['City','Restaurant','URL'])
restaurants_df

,City,Restaurant,URL


In [42]:
city_df = pd.read_csv("City_URL.csv")
city_df.head()

,City,URL
0,TP. HCM,https://www.foody.vn/ho-chi-minh
1,Hà Nội,https://www.foody.vn/ha-noi
2,Đà Nẵng,https://www.foody.vn/da-nang
3,Cần Thơ,https://www.foody.vn/can-tho
4,Khánh Hoà,https://www.foody.vn/khanh-hoa


# Functions

In [14]:
def scrollToPageEnd(driver):
    SCROLL_PAUSE_TIME = 0.5

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [61]:
def createRestaurantsDF(url, city):

    driver.get(url)

    time.sleep(wait_time)

    dummy0 = driver.find_element_by_xpath('//div[@class="nav-box"]')

    time.sleep(wait_time)

    scrollToPageEnd(driver)

    time.sleep(wait_time)

    dummy2 = driver.find_element_by_xpath('//*[@id="ajaxRequestDiv"]/div/div[2]')

    dummy3 = driver.find_elements_by_xpath('.//div[@class="content-item ng-scope"]')
    
    temp_df = pd.DataFrame(columns=['City','Restaurant','URL'])
    
    for elem in dummy3:
        record = {
            "City": city,
            "Restaurant": elem.find_element_by_xpath('.//div[@class="title fd-text-ellip"]').text,
            "URL": elem.find_element_by_xpath('.//div[@class="title fd-text-ellip"]/a').get_attribute('href')
        }
        temp_df = temp_df.append(record, ignore_index=True)
     
    return temp_df 


# Crawl

In [63]:
for index in range(0, city_df.index.max()+1):
    row = city_df.loc[index]
    temp_df = createRestaurantsDF(row['URL'], row['City'])
    
    restaurants_df = pd.concat([restaurants_df,temp_df], ignore_index=True)

restaurants_df

,City,Restaurant,URL
0,TP. HCM,Biscotti - Bánh Hạt Dinh Dưỡng - Huỳnh Văn Bánh,https://www.foody.vn/ho-chi-minh/biscotti-banh...
1,TP. HCM,2 Béo - Bánh Sầu Riêng - Shop Online,https://www.foody.vn/ho-chi-minh/2-beo-banh-sa...
2,TP. HCM,Bánh 9 Sạch - Bánh Sầu Riêng - Bắc Hải,https://www.foody.vn/ho-chi-minh/banh-9-sach-b...
3,TP. HCM,Biscotti - Bánh Hạt Dinh Dưỡng - Phú Thọ Hòa,https://www.foody.vn/ho-chi-minh/biscotti-banh...
4,TP. HCM,Bánh 9 Sạch - Bánh Sầu Riêng - Đường Số 7,https://www.foody.vn/ho-chi-minh/banh-9-sach-b...
...,...,...,...
4258,Bắc Kạn,Tô Hải Yến,https://www.foody.vn/bac-kan/to-hai-yen
4259,Lai Châu,Ding Tea - Trần Phú,https://www.foody.vn/lai-chau/ding-tea-tran-phu
4260,Lai Châu,Mộc Quán - Đường 58,https://www.foody.vn/lai-chau/moc-quan-duong-58
4261,Lai Châu,Vịt Cỏ Vân Đình,https://www.foody.vn/lai-chau/vit-co-van-dinh


In [65]:
restaurants_df.to_csv("Restaurants_URL.csv", index=False, header=True, columns=restaurants_df.columns)